# Dev 04 - Spectral Processing

Using the `toshi` module to read in data and process for spectra

In [ ]:
import os
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi
import cspec

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/20200615_010423.194421-8C-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

In [ ]:
zmap = cspec.colormap.zmap()
cmap = matplotlib.colors.LinearSegmentedColormap.from_list('colors', zmap[:, :3], N=len(zmap))

In [ ]:
s = time.time()
ray_pulses, cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

In [ ]:
# Dimentions
naz = len(ray_pulses)
ngate_long = ray_pulses[0][0].ngate_long_hi
ngate_short = ray_pulses[0][0].ngate_short_hi
ngate = ngate_long + ngate_short

# Elevation assumed to be flat from the very first pulse
scan_el = ray_pulses[0][0].elevation
scan_time = time.strptime(os.path.basename(file)[:15], '%Y%m%d_%H%M%S')

# Sampling code from the CPI header
fs = 1.0e6 * (1 << cpi_headers[0].fs_code)
dr = 3.0e8 / fs / 2
r = 1.0e-3 * np.arange(0, ngate, dtype=np.single) * dr

# Go through the pulses for azimuth
a = np.zeros((naz), dtype=np.single)
for k, pulses in enumerate(ray_pulses):
    a[k] = pulses[0].azimuth
    
# Choose ray 2 to whatever that completes the 360-deg coverage
n = np.argmin(np.abs(a[3:] - a[2])) + 1
a = a[2:n+2]

In [ ]:
# Noise estimate, try azimuth 0, around 20-25 km
ia = np.argmin(np.abs(a[3:] - 0.0)) + 1
ir, er = np.argmin(np.abs(r - 20.0)), np.argmin(np.abs(r - 25.0))
samples = np.zeros((len(ray_pulses[ia]), er-ir), dtype=np.csingle)

# Gather the samples. Ignore phase code since we are only interested in amplitude
for k, pulse in enumerate(ray_pulses[ia]):
    samples[k, :] = pulse.h_long_hi[ir:er]
noise = np.mean(np.abs(samples)) ** 2
print('Noise estimate in 16-bit ADU: {:.4f}'.format(noise))

In [ ]:
s = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)

# Go through the pulses
for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, ngate_short:] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
        p[j, :ngate_short] = pulse.h_short_hi * np.exp(-1j * pulse.phase_h_short)
    # pp = pp - np.mean(pp, axis=0)
    pp = p[1:, :] * np.conj(p[:-1, :])
    s[k, :] = np.mean(np.abs(p) ** 2, axis=0)
    v[k, :] = np.angle(np.sum(pp, axis=0))

# Signal
s -= noise
s[s <= 0] = 0.01
snr = 10 * np.log10(s / noise)
z = 10 * np.log10(s * (r + 0.5e-3 * dr) ** 2) - 60
# z[:, :ngate_short] -= 20

# Thresholding
m = snr < 0
z[m] = np.nan
v[m] = np.nan

# Choose ray 2 to whatever that completes the 360-deg coverage
v = v[2:n+2, :]
z = z[2:n+2, :]
s = s[2:n+2, :]
snr = snr[2:n+2, :]

In [ ]:
ce = np.cos(np.deg2rad(scan_el))
rr, aa = np.meshgrid(r, np.deg2rad(a))
xx = rr * ce * np.sin(aa)
yy = rr * ce * np.cos(aa)

sweep = {
    'longitude': 140.0440751,
    'latitude': 39.7294019,
    'scan_time': scan_time,
    'scan_el': scan_el,
    'aa': aa,
    'rr': rr,
    'z': z,
    'v': v,
    'm': m
}

In [ ]:
# turbines = toshi.turbine_cells

# a_turb = turbines[:, 0] / 180.0 * np.pi
# r_turb = turbines[:, 1]
# x_turb = r_turb * np.sin(a_turb)
# y_turb = r_turb * np.cos(a_turb)


plt.figure(figsize=(5.25, 4), dpi=200)
# plt.pcolormesh(xx, yy, z, cmap=cmap)
# plt.pcolormesh(xx[::2, :500], yy[::2, :500], z[::2, :500], cmap=cmap)
plt.pcolormesh(xx[:, :500], yy[:, :500], z[:, :500], cmap=cmap)
# plt.plot(x_turb, y_turb, 'xw')
# plt.clim((-32, 96))
plt.colorbar()
plt.grid()
# plt.xlim((0, 5))
# plt.ylim((-10.5, -5.5))
# plt.xlim((-5, 1))
# plt.ylim((4, 10))
plt.xlim((-20, 5))
plt.ylim((0, 25))

In [ ]:
# cells = cinterp.pos2cellid(turbines, a, r)